In [1]:
from IPython.display import HTML
import random

def hide_toggle(for_next=False):
    this_cell = """$('div.cell.code_cell.rendered.selected')"""
    next_cell = this_cell + '.next()'

    toggle_text = 'Toggle show/hide'  # text shown on toggle link
    target_cell = this_cell  # target cell to control with toggle
    js_hide_current = ''  # bit of JS to permanently hide code in current cell (only when toggling next cell)

    if for_next:
        target_cell = next_cell
        toggle_text += ' next cell'
        js_hide_current = this_cell + '.find("div.input").hide();'

    js_f_name = 'code_toggle_{}'.format(str(random.randint(1,2**64)))

    html = """
        <script>
            function {f_name}() {{
                {cell_selector}.find('div.input').toggle();
            }}

            {js_hide_current}
        </script>

        <a href="javascript:{f_name}()">{toggle_text}</a>
    """.format(
        f_name=js_f_name,
        cell_selector=target_cell,
        js_hide_current=js_hide_current, 
        toggle_text=toggle_text
    )

    return HTML(html)

hide_toggle()

### Policy Gradient Methods ###

In this notebook we will examine and implement various policy gradient methods for reinforcement learning.

Credits go to Seita's blog, John Schulman's notes and PhD thesis, and the CS234 lectures, from which the explanations in this notebook are synthesized. I used the CS234 assignment 3 startercode to focus more on the conceptual parts of implementing the learning algorithms over implementation parts like logging. 

### Vanilla Policy Gradient ###

We will first implement vanilla policy gradient (REINFORCE), a classic policy gradient reinforcement learning algorithm.

The goal of reinforcement learning is for an agent to learn to act in a dynamic environment so as to maximize its expected cumulative reward over the course of a time-horizon. Policy gradient methods solve the problem of control by directly learning the policy $\pi: \mathcal{S} \rightarrow \mathcal{A}$ from observations of rewards obtained by interacting with the environment. 



Formally define a trajectory $\tau$ as a tuple $(s_0, a_0, r_0, s_1, a_1, ..., r_T)$ denoting a sequence of state-action-rewards observed over the course of some episode of interaction with the agent's environment, and let $R(\tau)$ denote the finite-horizon return, aka cumulative sum of rewards. Then our goal is the maximize the _expected_ finite-horizon return where the expectation is over trajectories sampled from the stochastic policy $\pi_{\theta}$ (here we let $\theta$ denote the parameters of the policy $\pi$)--i.e.

\begin{equation}
max_{\theta} J(\pi_{\theta}) = \mathbb{E}_{\tau \sim \pi_{\theta}} [R(\tau)]
\end{equation}

In order to optimize $J$ using gradient ascent, we need an efficiently computable form of its gradient. First, let's compute an analytical form of the gradient, then see how we can approximate it with sampling.

We use the log-derivative trick to push a form of the gradient of the policy into the the gradient of $J$.

I liked Seita's explanation of it--"the log derivative trick tells us how to insert a log into an expectation when starting from $\nabla_{\theta} \mathbb{E}[f(x)]$"

\begin{align*}
\nabla_{\theta} \mathbb{E}[f(x)] &= \nabla_{\theta} \int p_{\theta}(x) f(x) dx \\
&= \int \dfrac{p_{\theta}(x)}{p_{\theta}(x)} \nabla_{\theta} p_{\theta}(x) f(x) dx \\
&= \int p_{\theta}(x) \nabla_{\theta} \log p_{\theta}(x) f(x) dx \\
&= \mathbb{E}[\nabla_{\theta} \log p_{\theta}(x) f(x)]
\end{align*}

In the gradient of $J$ we are concerned with the gradient of the log probability of trajectories, so we derive its form now.

\begin{align*}
\nabla_{\theta} \log p_{\theta}(\tau) &= \nabla_{\theta} \log \left( \mu(s_{0}) \prod_{t=0}^{T-1} \pi_{\theta}(a_t|s_t) P(s_{t+1|s_t,a_t}) \right) \\
&= \nabla_{\theta} \left( \log \mu_{s_0} + \sum_{t=0}^{T-1} (\log \pi_{\theta}(a_t | s_t) + \log P(s_{t+1} | s_t, a_t))\right) \\
&= \nabla_{\theta} \sum_{t=0}^{T-1} \log \pi_{\theta}(a_t | s_t) \\
\end{align*}

Putting the pieces together, the gradient of the expected return is

\begin{equation}
    \nabla_{\theta} J(\pi_{\theta}) = \mathbb{E}_{\tau \sim \pi_{\theta}} [ R(\tau) \sum_{t=0}^T \nabla_{\theta} \log \pi_{\theta}(a_t|s_t)]
\end{equation}

The log probability of each action is weighted by the rewards associated with it. The gradient has an intuitive interpretation--it encourages us to increase the probability of actions which lead to high expected return.

Conveniently, $\nabla_{\theta} J$ turns out to have the form of an expectation. Because of this, we can estimate it a la Monte Carlo methods using samples from our environment--i.e.

\begin{equation}
\hat{g} = \dfrac{1}{|\mathcal{D}|} \sum_{\tau \in \mathcal{D}} \sum_{t=0}^T \nabla_{\theta} \log \pi_{\theta}(a_t|s_t) R(\tau)
\end{equation}

Where $\mathcal{D}$ is a dataset of trajectories.

### Variance ###

An issue with the estimator $\hat{g}$ is that it has high variance. (One of the reasons for this, which we will go into more depth later, is that $R(\tau)$ sums up individual reward samples over long sequences.) If we have high variance in our estimate of the policy gradient, then the update will cause the policy to fluctuate across updates in ways that don't reflect the overall dynamics of the environment.

One way to decrease variance of the estimator starts by observing that the action taken at time $t$ affects only the rewards reaped at timestep $t$ onwards. This makes sense in terms of credit assignment, and also reduces the number of timesteps we sum over. Let's do some algebra to get the gradient into a form we want

\begin{align*}
\nabla_{\theta} \mathbb{E}_{\tau \sim \pi_{\theta}} [R (\tau)] &= \mathbb{E}_{\tau \sim \pi_{\theta}} [ R(\tau) \sum_{t=0}^T \nabla_{\theta} \log \pi_{\theta}(a_t|s_t)] \\
&= \mathbb{E}_{\tau \sim \pi_{\theta}} [ \sum_{t' = 0}^T r_{t'} \sum_{t=0}^T \nabla_{\theta} \log \pi_{\theta}(a_t|s_t)] \\
&= \mathbb{E}_{\tau \sim \pi_{\theta}} [ \sum_{t=0}^T \nabla_{\theta} \log \pi_{\theta}(a_t|s_t) (\sum_{t' = t}^T r_{t'})]  \\
\end{align*}

[Insert explanation here for algebra]

The literature likes to refer to $\sum_{t' = t}^T r_{t'}$ as rewards-to-go. We weight each action its causal consequence--the future rewards it impacts. We note that this form has a resemblance to Bellman forms of value, which concern the reward at a given time step, as well as the future consequences.

A second way to decrease the variance of the policy gradient estimate is by subtracting a baseline from the rewards-to-go. 

\begin{equation}
\nabla_{\theta} \mathbb{E}_{\tau \sim \pi_{\theta}} [R(\tau)] = \mathbb{E}_{\tau \sim \pi_{\theta}} [ \sum_{t=0}^T \nabla_{\theta} \log \pi_{\theta}(a_t|s_t) (\sum_{t' = t}^T r_{t'} - b(s_t))]
\end{equation}

An intuitive choice for $b(s_t)$ is the state-value function $V^{\pi}(s_t)$. When we use the state-value function as a baseline, we are choosing to weight an action by the difference between the rewards we got in our sample and the rewards we expected to get. If the difference is close to zero, then we shouldn't need to change our policy too much--i.e. the gradient for that state-action should be close to zero in the update.

There are a few ways to more formally explain why the baseline reduces the variance of the policy gradient estimator. I'll first summarize Seita's explanation, which is an approximate one.

\begin{align*}
Var(\hat{g}) &= Var(\sum_{t=0}^T \nabla_{\theta} \log \pi_{\theta}(a_t|s_t) (\sum_{t' = t}^T r_{t'})- b(s_t)) \\
&  \approx \mathbb{E}_{\tau \sim \pi_{\theta}} [(\sum_{t=0}^T \nabla_{\theta} \log \pi_{\theta}(a_t|s_t) (\sum_{t' = t}^T r_{t'})- b(s_t))^2] \\
& \approx \sum_{t=0}^{T-1} \mathbb{E}_{\tau \sim \pi_{\theta}} [(\log \pi_{\theta}(a_t|s_t) (\sum_{t' = t}^T r_{t'})- b(s_t))^2] \\
& \approx \sum_{t=0}^{T-1} \mathbb{E}_{\tau \sim \pi_{\theta}} [(\log \pi_{\theta}(a_t|s_t))^2] \mathbb{E}_{\tau \sim \pi_{\theta}}[(\sum_{t' = t}^T r_{t'})- b(s_t))^2] \\
\end{align*}

He first approximates the variance of a sum by the sum over timesteps of variances. Then he uses the formula $Var(X) = \mathbb{E}[X^2] - (\mathbb{E}[X])^2$ noting that $\mathbb{E}[X] = 0$ for the log probability form. [To-do: explanation and proof]. The third approximation comes from independence. Seita says that the approximations are fine since recent advances in RL, like A3C, break up correlation among samples.

Based on these approximations, we can see that that a well-chosen baseline can decrease the variance of $\hat{g}$.

I'm unsatisfied with these explanations, and plan on adding my own based off of the CS234 assignment to this notebook.

### Policy Network ###

The policy is represented as a multi-layer perceptron so that we can learn to act in environments with high dimensional states and/or continous action spaces.

In [2]:
from collections import OrderedDict
import torch
import numpy as np
import torch.nn as nn

def build_mlp(input_size, output_size, n_layers, size):    
    modules = OrderedDict()
    modules['Linear_Input'] = nn.Linear(input_size, size)
    modules['ReLU_Input'] = nn.ReLU()
    for i in range(n_layers):
        modules['Linear_'+str(i)] = nn.Linear(size, size)
        modules['ReLU_'+str(i)] = nn.ReLU()
    modules['Linear_Output'] = nn.Linear(size,output_size)
    sequential = nn.Sequential(modules)
    return sequential

# cartpole
observation_dim        = 4
n_layers               = 1
layer_size             = 64

policy_network = build_mlp(observation_dim, 1, n_layers, layer_size)

### Loss ###

Below is the implementation of the policy gradient update. Note here that advantage denotes $\sum_{t' = t}^T r_{t'}- V^{\pi}(s_t)$

In [3]:
import os, sys
os.chdir("/Users/annhe/vpg/code")

hide_toggle()

In [4]:
# from code/policy_gradient.py

def update_policy(self, observations, actions, advantages):
        """
        Args:
            observations: np.array of shape [batch size, dim(observation space)]
            actions: np.array of shape
                [batch size, dim(action space)] if continuous
                [batch size] (and integer type) if discrete
            advantages: np.array of shape [batch size]

        Perform one update on the policy using the provided data.
        To compute the loss, you will need the log probabilities of the actions
        given the observations. Note that the policy's action_distribution
        method returns an instance of a subclass of
        torch.distributions.Distribution, and that object can be used to
        compute log probabilities.
        See https://pytorch.org/docs/stable/distributions.html#distribution
        """
        observations = np2torch(observations)
        actions = np2torch(actions)
        advantages = np2torch(advantages)

        self.optimizer.zero_grad()
        res = self.policy.action_distribution(observations).log_prob(actions) 

        loss = -(res * advantages).mean()
        loss.backward()
        self.optimizer.step()

### Baseline Refitting ###

Note that we will have to update the baseline, $V^{\pi}(s_t)$ every time we perform a policy update. This is because we want our estimate of the value function to be as accurate as possible, given the data we have. To do so, we compute the mean squared error loss between a forward pass of the network (current value estimate) and the most recent returns we have.

In [5]:
def update_baseline(self, returns, observations):
        """
        Args:
            returns: np.array of shape [batch size], containing all discounted
                future returns for each step
            observations: np.array of shape [batch size, dim(observation space)]
        """
        returns = np2torch(returns)
        observations = np2torch(observations)
        
        self.optimizer.zero_grad()
        preds = self.forward(observations)
        loss = ((returns - preds)**2).mean()
        loss.backward()
        self.optimizer.step()

### Experiment ###

Now we are ready to run some experiments. We would hope to see that the introduction of the baseline improves performance over just using the returns in the policy gradient estimate. To do so, we experiment on three OpenAI gym environments--cartpole, pendulum, and half-cheetah. For each environment we choose three random seeds, and run the agent with and without baseline using those three seeds. We plot the average expected return for each experiment with error bars. 

The cartpole and pendulum results do not really demonstrate the difference between baseline and no baseline, but cheetah seems to be a complex enough environment to show the difference. In cheetah, we see that the baseline approach outperforms no baseline, especially at the very end, where the no baseline approach actual deteriorates in performance while the baseline approach continues to improve. Without a baseline, the policy gradient updates may be too large. In fact, even with a baseline, VPG is too "greedy" an approach. An intuitive explanation is that overcorrecting (changing the way you act too drastically) based on feedback may incur immediate benefits but prevent long term learning and generalization. Follow-up work like TRPO and PPO aim to make monotonic improvements in performance by constraining the difference between the policy before and after updates using KL-Divergence. 

### Empirical Results / Baseline vs no Baseline ###

<img src="code/results/results-cartpole-old.png">
<img src="code/results/results-pendulum.png">
<img src="code/results/results-cheetah.png">

### Monotonic Improvement and TRPO ###

As shown in the experiment above, VPG does not necessarily make monotonic improvements to the expected performance of a policy. TRPO constrains the step-size of the gradient updates to impose monotonic improvement. First we derive the theoretical basis for TRPO, then we look into the details and make approximations and estimates in order to implement it.

Instead of setting the optimization objective to be the expected performance of a policy, we instead set the objective to be a loss function whose monotonic improvement guaranteed monotonic improve of the policy itself. Consider the following identity due to Kakade and Langford,

\begin{equation}
\eta_{\pi}(\tilde{\pi}) = \eta_{\pi}(\pi) + \mathbb{E}_{\tau \sim \tilde{\pi}}[\sum_{t=0}^{\infty} \gamma^{t} A_{\pi}(s_{t}, a_{t})]
\end{equation}

Where $\eta_{\pi}(\pi)$ is the discounted sum of rewards from following $\pi$ and $A_{\pi}(s_{t}, a_{t})$ is the advantage of taking action $a_{t}$ over following $\pi$ at time step $t$. If we massage this equation into a slightly different form, we can begin to see the theoretical basis for TRPO.

Note that $\mathbb{E}_{\tau \sim \tilde{\pi}} = \mathbb{E}_{s_0, a_0, s_1, a_1, ... \sim \tilde{\pi}}$.

If we define $P(s_t = s | \tilde{\pi})$ to be the probability that we end up in state $s$ at time $t$ by following $\pi$, then we can conveniently rewrite $\eta_{\pi}(\tilde{\pi})$ as 

\begin{align*}
\eta_{\pi}(\tilde{\pi}) &= \eta_{\pi}(\pi) + \mathbb{E}_{s_0, a_0, s_1, a_1, ... \sim \tilde{\pi}}[\sum_{t=0}^{\infty} \gamma^{t} A_{\pi}(s_{t}, a_{t})] \\
&= \eta_{\pi}(\pi) + \sum_{t=0}^{\infty} \sum_{s} P(s_t = s | \tilde{\pi}) \sum_{a} \tilde{\pi}(a|s) \gamma^{t} \mathcal{A}_{\pi}(s,a) \\
&= \eta_{\pi}(\pi) +  \sum_{s} \sum_{t=0}^{\infty} P(s_t = s | \tilde{\pi}) \sum_{a} \tilde{\pi}(a|s) \gamma^{t} \mathcal{A}_{\pi}(s,a) \\
&= \eta_{\pi}(\pi) +  \sum_{s} p_{\tilde{\pi}}(s) \sum_{a} \tilde{\pi}(a|s) \gamma^{t} \mathcal{A}_{\pi}(s,a)
\end{align*}

Where $p_{\tilde{\pi}}(s) = P(s_0 = s | \tilde{\pi}) + \gamma \cdot P(s_1 = s | \tilde{\pi}) + ...$ is the discounted state visitation frequency. From the formulation above, we can see that any policy update that has positive expected advantage at every state is guaranteed to improve policy performance.

However, in our computation of the policy update, we cannot compute $p_{\tilde{\pi}}(s)$ as we do not have samples from the new policy $\tilde{\pi}$. We make a local approximation to $\eta$ by using $p_{\pi}(s)$ instead, yielding the modified objective

\begin{equation}
L_{\pi}(\tilde{\pi}) = \eta_{\pi}(\pi) +  \sum_{s} p_{\pi}(s) \sum_{a} \tilde{\pi}(a|s) \gamma^{t} \mathcal{A}_{\pi}(s,a)
\end{equation}

The first order gradient of $L_{\pi}(\tilde{\pi})$ and $\eta_{\pi}(\tilde{\pi})$ are equivalent, which means we can indirectly improve $\eta_{\pi}(\tilde{\pi})$ by directly optimizing $L_{\pi}(\tilde{\pi})$. However, we do not yet know how large a step size we can take and still have guaranteed improvement.

A theorem (cite) says

\begin{equation}
V^{\pi_{new}} \ge L_{\pi_{old}}(\pi_{new}) - \dfrac{2 \epsilon \gamma}{(1-\gamma)^2} \left( D_{TV}^{max}(\pi_{old}, \pi_{new}) \right)^2
\end{equation}

Since $(D_{TV})^2$ is upper bounded by KL divergence, we also get

\begin{equation}
V^{\pi_{new}} \ge L_{\pi_{old}}(\pi_{new}) - \dfrac{2 \epsilon \gamma}{(1-\gamma)^2} D_{KL}^{max}(\pi_{old}, \pi_{new}) 
\end{equation}

To see that we can get monotonic improvement by optimizing $L$,

Let $M_i(\pi) = L_{\pi_i}(\pi) - \dfrac{4 \epsilon \gamma}{(1-\gamma)^2} D_{KL}^{max}(\pi_{old}, \pi_{new})$

Note that $M_i(\pi_i) = L_{\pi_i}(\pi_i) =  V^{\pi_i}$

Then 

\begin{align*}
V^{\pi_{i+1}} & \ge L_{\pi_{i}}(\pi_{i+1}) - \dfrac{2 \epsilon \gamma}{(1-\gamma)^2} D_{KL}^{max}(\pi_{i}, \pi_{i+1}) \\
\end{align*}
And subtracting $V^{\pi_{i}}$ from both sides
\begin{align*}
V^{\pi_{i+1}} - V^{\pi_{i}} & \ge M_{i}(\pi_{i+1}) - M_{i}(\pi_{i})\\
\end{align*}
Since $M_i(\pi_i) =  V^{\pi_i}$.

Instead of optimizing $M$, which includes a penalty term for the KL divergence, TRPO optimizes against a constraint on the KL divergence.

### Sample-Based Estimation of the Objective and Constraint ###

### PPO vs VPG (baseline) ###

<img src="code/results/results-cartpole-ppo-vpg.png">
